In [3]:
%pip install qiskit
%pip install qiskit_machine_learning
%pip install qiskit_algorithms

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Data management
import numpy as np
import pandas as pd

# Maths
import math
from math import pi

# Plot
import matplotlib.pyplot as plt
import seaborn as sns

# ML
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

# Additional imports
import pylab as pl
from random import random
from numpy import linalg
from qiskit_machine_learning.datasets import ad_hoc_data
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute

# Plot configuration
%matplotlib inline
sns.set_theme()
sns.set_context("poster")
sns.set_style("ticks")

In [ ]:
class Perceptron_Hybrid:
    """
    Perceptron classifier trained using a hybrid approach of classical and quantum search.

    Parameters:
    -----------
    separators : array-like
        Set of separators.
    nb_ampli : int, optional
        Amplification parameter for quantum search. Defaults to 10.
    quantum : bool, optional
        If True, use quantum search for finding the best separator. Defaults to False.
    opti : bool, optional
        If True, use optimized Grover's algorithm for quantum search. Defaults to False.
    """

    def __init__(self, nb_hyperplanes, nb_ampli=10, quantum=False, opti=False):
        self.separators = None  # Set of separators
        self.nb_hyperplanes = nb_hyperplanes
        self.selected = 0  # The selected separator
        self.n_iter_ = 0  # Number of steps
        self.coef_ = None  # Default hyperplane
        self.quantum = quantum  # If we use the quantum search
        self.nb_ampli = nb_ampli  # The amplification parameter for the quantum search
        self.opti = opti  # If we use the opti Grover

    def fit(self, X, y):
        """
        Train the perceptron classifier.

        Parameters:
        -----------
        X : array-like of shape (n_samples, n_features)
            Training data.
        y : array-like of shape (n_samples,)
            Target labels.

        Returns:
        --------
        int
            The number of iterations.

        One of the "separators" will be chosen.
        """
        if self.separators is None:
            len_data = len(X[0])
            self.separators = np.random.multivariate_normal([0]*len_data,np.eye(len_data),size=self.nb_hyperplanes)
            np.random.shuffle(self.separators)
            self.coef_ = self.separators[self.selected]
            
        X_ = np.array([[j for j in i] for i in X])
        y_ = np.array([1 if i == 1 else -1 for i in y])  # Security to ensure that the classes are {-1,1} and not {0,1}.

        for k in range(len(self.separators)):  # For each separator
            # Oracle over the points
            oracle = [int(y_[i] * X_[i, :].dot(self.separators[k]) <= 0) for i in range(len(y_))]
            # Search
            m, steps = classical_search(oracle) if not self.quantum else quantum_search(oracle, nb_ampli=self.nb_ampli, opti=self.opti)

            self.n_iter_ += steps

            if y_[m] * X_[m, :].dot(self.separators[k]) > 0:  # If successful we choose this hyperplane
                self.selected = k
                self.coef_ = self.separators[self.selected]
                break

        return self.n_iter_

    def predict(self, X):
        """
        Predict class labels for samples in X.

        Parameters:
        -----------
        X : array-like of shape (n_samples, n_features)
            Samples.

        Returns:
        --------
        array-like of shape (n_samples,)
            Predicted class labels.
        """
        if self.separators is None:
            len_data = len(X[0])
            self.separators = np.random.multivariate_normal([0]*len_data,np.eye(len_data),size=self.nb_hyperplanes)
            np.random.shuffle(self.separators)
            self.coef_ = self.separators[self.selected]
        return np.array([1 if x.dot(self.separators[self.selected]) > 0 else -1 for x in X])